In [4]:
import nibabel as nib
import numpy as np
from glob import glob
import shutil

In [8]:
images = sorted(glob("../input/train/v3/mri/*.gz"))
labels = sorted(glob("../input/train/v3/masks/*.nii"))

print("Total MRIs: {}".format(len(images)))
print("Total masks: {}".format(len(labels)))

Total MRIs: 102
Total masks: 101


In [12]:
for mri, mask in zip(images, labels):
    if mri.split("/")[-1].split(".")[0] != mask.split("/")[-1].split(".")[0]:
        print(mri, mask)

mris, masks = [], []

for mri, mask in zip(images, labels):
    mris.append(mri.split("/")[-1].split(".")[0])
    masks.append(mask.split("/")[-1].split(".")[0])
    
print(len(mris))
print(len(masks))

mri_not_in_mask = [mri for mri in mris if mri not in masks]
mask_not_in_mri = [mask for mask in masks if mask not in mris]

print(len(mri_not_in_mask), len(mask_not_in_mri))

idx = 0
for mri, mask in zip(mris, masks):
    if mri == mask:
        
        mask_old = f"../input/train/v3/masks/{mask}.nii"
        mask_new = f"../input/train/v3/new/masks/{idx}_{mask}.nii"

        mri_old  = f"../input/train/v3/mri/{mri}.nii.gz"
        mri_new  = f"../input/train/v3/new/mri/{idx}_{mri}.nii.gz"
        
        shutil.move(mask_old, mask_new)
        shutil.move(mri_old,  mri_new)
        
        idx += 1
    else:
        print("Error on file: ".format(mri))

In [23]:
from monai.transforms import (
    AddChanneld,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
)


In [24]:
loader = LoadImage(dtype=np.float32)

In [27]:
image, metadata = loader(images[0])
print(f"input: {images[0]}")
print(f"image shape: {image.shape}")
print(f"image affine:\n{metadata['affine']}")
print("")
print(f"image pixdim:\n{metadata['pixdim']}")

input: ../input/train/v3/mri/0_20120626155116_3.nii.gz
image shape: (512, 512, 29)
image affine:
[[-5.82221925e-01  6.54081181e-02  5.77459261e-02  1.98172562e+02]
 [ 6.54087290e-02  5.82237542e-01 -6.48713298e-03 -1.70069733e+02]
 [ 4.25565708e-03  0.00000000e+00  8.00000095e+00 -9.84200287e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

image pixdim:
[-1.        0.5859    0.5859    8.000213  0.666664  0.        0.
  0.      ]


In [29]:
metadata["pixdim"]

array([-1.      ,  0.5859  ,  0.5859  ,  8.000213,  0.666664,  0.      ,
        0.      ,  0.      ], dtype=float32)

In [22]:
example.header.get_zooms()

(0.5859, 0.5859, 8.000213)